In [1]:
import pandas

In [2]:
households = pandas.read_csv('data/household.csv',index_col='ID',na_values=['0'])
members = pandas.read_csv('data/members.csv',index_col=False)

#Total Households Surveyed: 257
#Total Members Surveyed: 961

In [34]:
print len(households) #total surveyed
print len(members) #total members surveyed

257
961


#Duration of Stay in PPR

In [4]:
households['Tinggal di PPR semenjak'].median()

9.0

In [5]:
households['Tinggal di PPR semenjak'].mean()

8.822314049586776

#Marital Status

Single mothers: 73
Single fathers: 8

Note none head of households were not filtered out for this, so lots of single members which do not equal number of households

In [6]:
members['Taraf Perkahwinan'].value_counts()

Bujang          438
Berkahwin       293
Ibu tunggal      73
Bapa tunggal      8
Name: Taraf Perkahwinan, dtype: int64

#Households with OKU members

Total: 20

In [7]:
members['oku_bool'] = members['Ahli dalam kategori OKU'].notnull()

In [8]:
oku_households = members[['ID','oku_bool']].to_dict(orient='records')

In [9]:
oku_household = {}
for i in oku_households:
      if not oku_household.has_key(i['ID']):
        if i['oku_bool']==True:
            oku_household[i['ID']]=True

In [10]:
len(oku_household)

20

#Vaccination

##Households that know kids are not vaccinated: 58
##Households that don't know kids vaccination status: 46

In [11]:
household_tidak_tahu_vaksinasi = households.loc[households['Adakah anda mempunyai kanak-kanak bawah umur 12 yang belum terima vaksinasi?'] == 'Tidak tahu']
household_tidak_vaksinasi = households.loc[households['Adakah anda mempunyai kanak-kanak bawah umur 12 yang belum terima vaksinasi?'] == 'Ya']

In [12]:
household_tidak_vaksinasi['Adakah anda mempunyai kanak-kanak bawah umur 12 yang belum terima vaksinasi?'].count()

58

In [13]:
household_tidak_tahu_vaksinasi['Adakah anda mempunyai kanak-kanak bawah umur 12 yang belum terima vaksinasi?'].count()

46

#Crime

##Households that have been broken into: 21
##Households that report drug activity in area: 206

In [14]:
households['Adakah rumah anda pernah dipecah masuk?'].value_counts()

Tidak    233
Ya        21
Name: Adakah rumah anda pernah dipecah masuk?, dtype: int64

In [15]:
households['Adakah terdapat aktiviti yang berkaitan dengan dadah di kawasan PPR?'].value_counts()

Ya       206
Tidak     46
Name: Adakah terdapat aktiviti yang berkaitan dengan dadah di kawasan PPR?, dtype: int64

#Education
##Households U17 that don't go to school: 4

In [16]:
kids_in_school = members[['ID','Kanak-kanak berumur 17 tahun ke bawah yang tidak bersekolah']].to_dict(orient='records')

In [24]:
not_in_school = []
for i in kids_in_school:
    if i['Kanak-kanak berumur 17 tahun ke bawah yang tidak bersekolah'] == "Ada":
        print i['Kanak-kanak berumur 17 tahun ke bawah yang tidak bersekolah']

Ada
Ada
Ada
Ada


#Youth between ages 15-21 that are working: 14


In [26]:
youth_age_mask = (members['Umur'] > 15) & (members['Umur'] < 21) 

In [40]:
youth_members = members[youth_age_mask]

print pandas.Series(youth_members['Adakah anda bekerja?']=="Ya").value_counts()

youth_working_mask = (youth_members['Adakah anda bekerja?']=="Ya")
youth_working = youth_members[youth_working_mask]

pandas.Series(youth_working[['ID','Jenis pekerjaan']])



False    95
True     14
Name: Adakah anda bekerja?, dtype: int64


ValueError: cannot copy sequence with size 2 to array axis with dimension 14

#Health

## Rat urine disease = No Household affected
## Dengue Fever: 48 households

In [72]:
households['Adakah anda mengenali mana-mana ahli keluarga yang jatuh sakit kerana denggi?'].value_counts()

Tidak    208
Ya        48
Name: Adakah anda mengenali mana-mana ahli keluarga yang jatuh sakit kerana denggi?, dtype: int64

In [133]:
#Household has Internet
households['Internet (unifi/maxis/lain-lain)'].notnull().value_counts() 

False    230
True      27
Name: Internet (unifi/maxis/lain-lain), dtype: int64

#Welfare

## 51 households getting BR1M or other form of welfare

In [91]:
#households recieving aid

welfare_recipients_mask = \
(members['Adakah anda pernah menerima Bantuan Rakyat 1Malaysia (BR1M)?'] == 'Ya') & \
(members['Adakah anda menerima sebarang bantuan kewangan bulanan daripada mana-mana agensi berikut?'].notnull())

welfare_recipients = members[welfare_recipients_mask]

len(welfare_recipients['ID'].unique())

#TODO generate DF of unique 

51

#Household racial breakdown

    Melayu              147
    India                40
    Cina                 12
    Bumiputera Sabah      4
    Orang asal            1
    siam                  1
    
#Household religious breakdown

    Islam       154
    Hindu        32
    Buddha       12
    Kristian      8

#Household state of origin breakdown

    Selangor - 10               55
    Perak - 08                  52
    Wilayah Persekutuan - 14    17
    Kedah - 02                  13
    Kelantan - 03               12
    Negeri Sembilan - 05        12
    Sabah - 12                  10
    Johor - 01                   7
    Lain-lain - 82               6
    Melaka - 04                  5
    Pahang - 06                  4
    Pulau Pinang - 07            4
    Terengganu - 11              4
    Sarawak - 13                 3
    Perlis - 09                  2


In [110]:
#household race breakdown
#assuming first member is representative


members_racial = members.where((pandas.notnull(members)), None)

members_racial = members_racial.to_dict(orient='records')

houses = {}
for member in members_racial:
    if member['Bangsa']:
        if not houses.has_key(member['ID']):
            houses[member['ID']] = [member['Bangsa']]
        else:
            houses[member['ID']].append(member['Bangsa'])

In [134]:
household_race = []
for k,v in houses.iteritems():
    household_race.append(v[0])
    
household_race = pandas.DataFrame(household_race,columns=['race'])

#2 mixed households siam+cina and melayu+bumiputera sabah

household_race['race'].value_counts()

Melayu              147
India                40
Cina                 12
Bumiputera Sabah      4
Orang asal            1
siam                  1
Name: race, dtype: int64

In [152]:
members_religion = members.where((pandas.notnull(members)), None)
members_religion = members_religion.to_dict(orient='records')

houses = {}
for member in members_religion:
    if member['Agama']:
        if not houses.has_key(member['ID']):
            houses[member['ID']] = member['Agama']
        else:
            pass

pandas.Series(houses).value_counts()

Islam       154
Hindu        32
Buddha       12
Kristian      8
dtype: int64

In [156]:
members_state = members.where((pandas.notnull(members)), None)
members_state = members_state.to_dict(orient='records')

houses = {}
for member in members_state:
    if member['Negeri lahir']:
        if not houses.has_key(member['ID']):
            houses[member['ID']] = member['Negeri lahir']
        else:
            pass
        
pandas.Series(houses).value_counts()

Selangor - 10               55
Perak - 08                  52
Wilayah Persekutuan - 14    17
Kedah - 02                  13
Kelantan - 03               12
Negeri Sembilan - 05        12
Sabah - 12                  10
Johor - 01                   7
Lain-lain - 82               6
Melaka - 04                  5
Pahang - 06                  4
Pulau Pinang - 07            4
Terengganu - 11              4
Sarawak - 13                 3
Perlis - 09                  2
dtype: int64

# Minors per Household (U18)

## Mean 2.4
## Median 2.0

In [176]:
children_df = members[members['Umur'] < 18]

children_per_household = children_df.groupby('ID').size()

print children_per_household.mean()
print children_per_household.median()

2.43790849673
2.0


# Residents per household
## mean: 4.5
## median: 4
## max: 12

In [186]:
print members.groupby('ID').size().mean()
print members.groupby('ID').size().median()
print members.groupby('ID').size().max()

4.5980861244
4.0
12


# Household Income (MYR)

    Mean:   1814.5
    Median: 1500
    Max:    6500
    Min:     400

In [210]:
#household income

household_income  = members.groupby('ID')['Pendapatan'].sum()

print household_income.mean()
print household_income.median()
print household_income.max()
print household_income.min()

1814.57635468
1500.0
6600.0
400.0


#Household Expenditure (MYR)

Mean: 1546.13
Median: 1400

In [211]:
members['total_expenditure'] = members['Perbelanjaan Peribadi'] + members['Penyumbang kepada isi rumah']

In [223]:
household_expenditure = members.groupby('ID')['total_expenditure'].sum()
print household_expenditure.mean()
print household_expenditure.median()

1546.13095238
1400.0


#Household Head Age

Mean: 65yro
Median: 46yro

In [248]:
#household head

members_household_heads = members.where((pandas.notnull(members)), None)

members_household_heads = members_household_heads.to_dict(orient='records')

houses = {}
for member in members_household_heads:
    if member['Umur']:
        if not houses.has_key(member['ID']):
            houses[member['ID']] = [member['Umur']]
        else:
            houses[member['ID']].append(member['Umur'])

household_head_age = []
for k,v in houses.iteritems():
    household_head_age.append(v[0])
    
print pandas.Series(household_head_age).mean()
print pandas.Series(household_head_age).median()

64.9215686275
46.0


#Jobs

In [273]:
#jobs

houses = {}
for member in members_household_heads:
    if member['Jenis pekerjaan']:
        if not houses.has_key(member['ID']):
            houses[member['ID']] = [member['Jenis pekerjaan']]
        else:
            houses[member['ID']].append(member['Jenis pekerjaan'])
            
pekerjaan_ketua_rumah = []
for k,v in houses.iteritems():
    pekerjaan_ketua_rumah.append(v[0])

    
pekerjaan_ketua_rumah = [ x.replace('driver lorry','pemandu lori') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('driver van','pemandu lori') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('pemandu van','pemandu lori') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('kontrak','kerja kontrak') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('kerja kerja kontrak','kerja kontrak') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('driver taxi','pemandu teksi') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('pemandu teksi part time','pemandu teksi') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('driver teksi','pemandu teksi') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('jaga budak','pengasuh') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('pengasuh kanak-kanak','pengasuh') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('kerja kilang','pekerja kilang') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('operator kilang','pekerja kilang') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('security','pengawal keselamatan') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('securiti','pengawal keselamatan') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('pengawal keselamatan guard','pengawal keselamatan') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('cashier speedmart','juruwang') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('kedai 7Eleven','juruwang') for x in pekerjaan_ketua_rumah ]
pekerjaan_ketua_rumah = [ x.replace('sales promoter','promoter') for x in pekerjaan_ketua_rumah ]

pandas.set_option('display.max_rows',len(pekerjaan_ketua_rumah))

pandas.Series(pekerjaan_ketua_rumah).value_counts()


pemandu lori                      23
cleaner                           13
pengawal keselamatan              13
kerja kontrak                      9
kilang                             7
pemandu teksi                      7
sekuriti                           5
berniaga                           5
kerani                             4
pengasuh                           4
pekerja kilang                     4
promoter                           3
juruwang                           3
surirumah                          3
pomen                              2
kelindan lori                      2
penghantar surat                   2
jurujual                           2
swasta                             2
despatch                           2
hotel                              2
mekanik                            2
pembantu jualan                    2
technician                         2
pencen askar                       1
welder                             1
driver                             1
j

#Education Level of Household Heads

In [291]:
#education level of household heads

houses = {}
for member in members_household_heads:
    if member['Tahap pendidikan']:
        if not houses.has_key(member['ID']):
            houses[member['ID']] = [member['Tahap pendidikan']]
        else:
            houses[member['ID']].append(member['Tahap pendidikan'])
            
pendidikan_ketua_rumah = []
for k,v in houses.iteritems():
    pendidikan_ketua_rumah.append(v[0].upper())
    
pendidikan_ketua_rumah = [ x.replace('FORM 3','SEKOLAH MENENGAH') for x in pendidikan_ketua_rumah ]    
pendidikan_ketua_rumah = [ x.replace('N','SEKOLAH RENDAH') for x in pendidikan_ketua_rumah ]    
pendidikan_ketua_rumah = [ x.replace('SEK REN','SEKOLAH RENDAH') for x in pendidikan_ketua_rumah ]    
pendidikan_ketua_rumah = [ x.replace('SEKOLAH RENDAHDAH','SEKOLAH RENDAH') for x in pendidikan_ketua_rumah ]    
pendidikan_ketua_rumah = [ x.replace('FORM 2','SEKOLAH MENENGAH') for x in pendidikan_ketua_rumah ]    
pendidikan_ketua_rumah = [ x.replace('S. MEN','SEKOLAH MENENGAH') for x in pendidikan_ketua_rumah ]    
pendidikan_ketua_rumah = [ x.replace('SEK MEN TINGKATAN','SEKOLAH MENGENAH') for x in pendidikan_ketua_rumah ]    
pendidikan_ketua_rumah = [ x.replace('TINGKATAN 2 2','SEKOLAH MENENGAH') for x in pendidikan_ketua_rumah ]    

pandas.Series(pendidikan_ketua_rumah).value_counts()

SPM                                                           68
SRP                                                           24
PMR                                                           15
DARJAH 6                                                      10
STPM                                                           9
DIPLOMA                                                        8
UPSR                                                           5
SEKOLAH MESEKOLAH RENDAHESEKOLAH RENDAHGAH                     4
DEGREE                                                         3
TISEKOLAH RENDAHGKATASEKOLAH RENDAH 2                          3
FORM 4                                                         3
DARJAH 4                                                       2
TISEKOLAH RENDAHGKATASEKOLAH RENDAH 5                          2
SEKOLAH RESEKOLAH RENDAH                                       2
FORM 1                                                         2
4                        